In [ ]:
import jieba
from tqdm import tqdm
import os
import torch
import torch.nn as nn
import random
import re
import time
import numpy as np

In [ ]:
def read_text(path):
    # root_dir = './jyxstxtqj_downcc.com/'
    # path = os.path.join(root_dir, '射雕英雄传.txt')
    with open(os.path.abspath(path), "r", encoding='ansi') as file:
        file_content = file.read()

    file_content = file_content. \
        replace("本书来自www.cr173.com免费txt小说下载站", '')
    file_content = file_content. \
        replace("更多更新免费电子书请关注www.cr173.com", '')
    file_content = file_content. \
        replace("----〖新语丝电子文库(www.xys.org)〗", '')

    file_content = file_content.replace("\n", '')
    file_content = file_content.replace("\t", '')
    file_content = file_content.replace(" ", '')
    file_content = file_content.replace('\u3000', '')
    file_content = file_content.replace('」', '”')
    file_content = file_content.replace('「', '“')
    file_content = file_content.replace('□', '')
    file_content = file_content.replace('』', '’')
    file_content = file_content.replace('『', '‘')
    # print(type(file_content))
    # print(len(file_content))
    text = list(jieba.cut(file_content))
    print('经过预处理后文章总词数：', len(text))
    return text

In [ ]:
word_lst = read_text(path='./jyxstxtqj_downcc.com/射雕英雄传.txt')

# 构建字典映射每个中文字到索引的关系
word2index = {}

for word in word_lst:
    if word not in word2index:
        word2index[word] = len(word2index)

index2word = {index: word for word, index in word2index.items()}

# 将中文转换为索引
index_lst = [word2index[word] for word in word_lst]

经过预处理后文章总词数： 585982


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        
    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.lstm(embedded)
        outputs = self.fc_out(outputs)

        return outputs

In [ ]:
vocab_size = len(word2index) # 词典中总共的词数
embed_size = 30 # 每个词语嵌入特征数
hidden_size = 512 # LSTM的每个时间步的每一层的神经元数量
num_layers = 2 # LSTM的每个时间步的隐藏层层数

max_epoch = 30
batch_size = 16
learning_rate = 0.001
sentence_len = 20
train_lst = [i for i in range(0,10000)]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = LSTM(vocab_size, embed_size, hidden_size, num_layers, vocab_size).to(device)
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### 模型训练
LSTM模型的训练需要注意一点，模型的目标输出语序列应该是输入序列的往后一个词语的序列，即每个输入序列向右移动一个位置作为目标序列，这样才能保证经过训练以后的模型可以根据输入依次生成词语。

同时在训练结束后需要保存模型以供测试

In [ ]:
for epoch in tqdm(range(max_epoch)):
    for i in train_lst:  # 打印循环中的进度条
        inputs = torch.tensor([index_lst[j:j+sentence_len] for j in range(i,i+batch_size)]).to(device)
        targets = torch.tensor([index_lst[j+1:j+1+sentence_len] for j in range(i,i+batch_size)]).to(device)
        outputs = model(inputs)

        loss = criterion(outputs.view(outputs.size(0)*outputs.size(1), -1), targets.view(-1))
        model.zero_grad()  # 梯度清零
        loss.backward()  # 反向传播
        optimizer.step()
        
'''保存模型'''
save_path = './trained_model/lstm2.pth'
torch.save(model, save_path)


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

In [ ]:
def tensor_to_str(index2word_dict, class_tensor):
    # 将张量转换为字符串
    class_lst = list(class_tensor)
    words = [index2word_dict[int(index)] for index in class_lst]
    
    # 将列表中的词语连接为一个字符串
    sentence = ''.join(words)
    return sentence

In [ ]:
test_model = torch.load('./trained_model/lstm2.pth').to('cpu')

generate_length = 100
test_set = [index_lst[i:i+sentence_len] for i in range(20000, 30000, 1000)]
target_set = [index_lst[i:i+sentence_len+generate_length] for i in range(20000, 30000, 1000)]


for i in range(0, len(test_set)):
    generate_lst = []
    generate_lst.extend(test_set[i])
    for j in range(0, generate_length):
        inputs = torch.tensor(generate_lst[-sentence_len:]) # 选取生成词语列表的最后sentence_len个元素作为下一次模型的输入
        outputs = test_model(inputs)

        predicted_class = torch.argmax(outputs, dim=-1)
        
        generate_lst.append(int(predicted_class[-1]))

    input_sentence = tensor_to_str(index2word, test_set[i])
    generate_sentence = tensor_to_str(index2word, generate_lst)
    target_sentence = tensor_to_str(index2word, target_set[i])
    # 打印生成结果
    print('测试结果', i)
    print()
    print('初始输入句：\n',input_sentence)
    print()
    print('模型生成句：\n',generate_sentence)
    print()
    print('期待生成句：\n',target_sentence)
    print()
    print('='*50)

测试结果 0

初始输入句：
 ，但见那乘马奔到大街转弯角处，忽然站住。完颜洪烈又是一奇，心想马匹

模型生成句：
 ，但见那乘马奔到大街转弯角处，忽然站住。完颜洪烈又是一奇，心想马匹，就留给两个还没出世，忽然转念：“别鬼使神差的，偏偏有人这时过来撞见。”鼓起勇气，过去拉那尸首，想拉入草丛之中藏起，再去叫丈夫。不料她伸手一拉，那尸首又呻吟了一下，声音甚是微弱。她才知此人未死。定睛看时，见他背后肩头中了一枝狼牙利箭，深入肉里，箭枝上染满了血污。天空雪花兀自不断飘下，那人全身已罩上了薄薄一层白雪，

期待生成句：
 ，但见那乘马奔到大街转弯角处，忽然站住。完颜洪烈又是一奇，心想马匹疾驰，必须逐渐放慢脚步方能停止，此马竟能在急行之际斗然收步，实是前所未睹，就算是武功高明之人，也未必能在发力狂奔之时如此神定气闲的蓦地站定。只见那矮胖子飞身下马，钻入一家店内。完颜洪烈快步走将过去，只见店中直立着一块大木牌，写着“太白遗风”四字，却是一家酒楼，再抬头看时，楼头一块极大的金字招牌，写着“醉仙楼”三个大字

测试结果 1

初始输入句：
 。那人腰里插了一柄砍柴用的短斧，斧刃上有几个缺口。两人刚

模型生成句：
 。那人腰里插了一柄砍柴用的短斧，斧刃上有几个缺口。两人刚笑道：“说不定！”这时了五年龙廷，那人全身已罩上了薄薄一层白雪，只须过得半夜，便冻也冻死了。她自幼便心地仁慈，只要见到受了伤的麻雀、田鸡、甚至虫豸蚂蚁之类，必定带回家来妥为喂养，直到伤愈，再放回田野，若是医治不好，就会整天不乐，这脾气大了仍旧不改，以致屋子里养满了诸般虫蚁、小禽小兽。她父亲是个屡试不第的村

期待生成句：
 。那人腰里插了一柄砍柴用的短斧，斧刃上有几个缺口。两人刚坐定，楼下脚步声响，上来两人。那渔女叫道：“五哥、六哥，你们一齐来啦。”前面一人身材魁梧，少说也有二百五六十斤，围着一条长围裙，全身油腻，敞开衣襟，露出毛茸茸的胸膛，袖子卷得高高的，手臂上全是寸许长的黑毛，腰间皮带上插着柄尺来长的尖刀，瞧模样是个杀猪宰羊的屠夫。后面那人五短身材，头戴小毡帽，白净面皮，手里提了一杆秤，

测试结果 2

初始输入句：
 查觉，当即带同亲随，由临安府的捕快衙役领路，亲自追拿刺客。追

模型生成句：
 查觉，当即带同亲随，由临安府的捕快衙役领路，亲自追拿刺客。追她，只听在断墙残瓦的破败之地，直到人挟持，当下捡出丈夫